## Modeling with CNN for Wildlife Image Classification

The data for this project is sourced from https://www.kaggle.com/datasets/akash2907/bird-species-classification 

### 1. Import Packages and Load Data

In [1]:
# Import necessary modules
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import imageio.v2 as imageio
import cv2


In [2]:
import keras
from keras.models import Sequential
from keras.utils import to_categorical
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

from keras.preprocessing.image import ImageDataGenerator
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping


### 2. Build model 

Loading the training dataset and applying augmentations using ImageDataGenerator. We'll need to one-hot-encode the target variable, which is done within the generator.

In [3]:
# set path and folder names
train_path = "../jpeg/train"
test_path = "../jpeg/test"

folder_names = os.listdir(train_path)[1:]

In [4]:
# Set the image and batch size
image_size = (200, 200)
batch_size = 32

# Number of classes
num_classes = 16

# Set early stopping if accuracy stops improving
early_stopping_monitor = EarlyStopping(monitor='val_loss', patience=3)

#### i. Base model -- create a model with very little augmentation or preprocessing

In [24]:
# Create an ImageDataGenerator for data augmentation and preprocessing
datagen = ImageDataGenerator(
    rescale=1.0 / 255,      # Normalize pixel values to [0, 1]
    validation_split=0.2    # Split the data into 80% for training and 20% for validation
)

# Load and preprocess images from the directory
train_generator = datagen.flow_from_directory(
    train_path,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',  # Set to 'categorical' for one-hot encoded labels
    subset='training'          # Use the training subset of your data
)

validation_generator = datagen.flow_from_directory(
    train_path,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',  # Set to 'categorical' for one-hot encoded labels
    subset='validation'        # Use the validation subset of your data
)


Found 123 images belonging to 16 classes.
Found 26 images belonging to 16 classes.


In [25]:
# Create the model
model1 = Sequential()

# Add a 2D Convolutional layer -- start with a small kernel size
model.add(Conv2D(32, kernel_size=(5, 5), padding='valid', activation='relu', input_shape=(image_size[0], image_size[1], 3)))

# Add a MaxPooling layer
model.add(MaxPooling2D(pool_size=(2, 2)))

# Add more Conv2D and MaxPooling2D layers as needed later ...

# Flatten the output from Convolutional layers
model.add(Flatten())

# Add a fully connected Dense layer
model.add(Dense(128, activation='relu'))

# Add the final Dense layer with softmax activation (ensures predictions sum to 1)
model.add(Dense(num_classes, activation='softmax'))


# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Summary of the model architecture
model.summary()


Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 196, 196, 32)      2432      
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 98, 98, 32)        0         
 g2D)                                                            
                                                                 
 flatten_2 (Flatten)         (None, 307328)            0         
                                                                 
 dense_4 (Dense)             (None, 128)               39338112  
                                                                 
 dense_5 (Dense)             (None, 16)                2064      
                                                                 
Total params: 39342608 (150.08 MB)
Trainable params: 39342608 (150.08 MB)
Non-trainable params: 0 (0.00 Byte)
__________

In [26]:
# Train the model
model.fit(
    train_generator,
#    steps_per_epoch=train_generator.samples // batch_size,
    epochs=20,
    validation_data=validation_generator,
#    validation_steps=validation_generator.samples // batch_size,
)


Epoch 1/20
4/4 [==============================] - 17s 4s/step - loss: 37.0858 - accuracy: 0.0732 - val_loss: 24.7217 - val_accuracy: 0.1923
Epoch 2/20
4/4 [==============================] - 15s 4s/step - loss: 15.0335 - accuracy: 0.1626 - val_loss: 6.4616 - val_accuracy: 0.1538
Epoch 3/20
4/4 [==============================] - 15s 4s/step - loss: 3.6674 - accuracy: 0.2276 - val_loss: 2.8848 - val_accuracy: 0.1154
Epoch 4/20
4/4 [==============================] - 14s 4s/step - loss: 2.1432 - accuracy: 0.3902 - val_loss: 2.6410 - val_accuracy: 0.1538
Epoch 5/20
4/4 [==============================] - 14s 4s/step - loss: 1.4997 - accuracy: 0.6911 - val_loss: 3.1919 - val_accuracy: 0.2692
Epoch 6/20
4/4 [==============================] - 14s 4s/step - loss: 0.9406 - accuracy: 0.8293 - val_loss: 3.0069 - val_accuracy: 0.3077
Epoch 7/20
4/4 [==============================] - 15s 4s/step - loss: 0.5224 - accuracy: 0.9593 - val_loss: 3.2543 - val_accuracy: 0.3077
Epoch 8/20
4/4 [===============

After 18 epochs, the model accuracy is 1.00 but the validation dataset accuracy is very low (0.3077). This is a small image dataset, and this suggests that the model is overfitting the data. We'll try out data augmentation next and see how the model does. 

#### ii. Model 2 -- create a model using augmentation and preprocessing
Use the ImageDataGenerator that augments the data by randomly shifting, rotating, and flipping the images. We'll use the same base model on this augmented data to see how it does. 

In [7]:
# Create an ImageDataGenerator for data augmentation and preprocessing
datagen = ImageDataGenerator(
    rescale=1.0 / 255,      # Normalize pixel values to [0, 1]
    rotation_range=20,      # Randomly rotate images within the range of 20 degrees
    width_shift_range=0.1,  # Randomly shift images horizontally within 10% of the image width
    height_shift_range=0.1, # Randomly shift images vertically within 10% of the image height
    horizontal_flip=True,   # Randomly flip images horizontally
    validation_split=0.2    # Split the data into 80% for training and 20% for validation
)

# Load and preprocess images from the directory
train_generator = datagen.flow_from_directory(
    train_path,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',  # Set to 'categorical' for one-hot encoded labels
    subset='training'          # Use the training subset of your data
)

validation_generator = datagen.flow_from_directory(
    train_path,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',  # Set to 'categorical' for one-hot encoded labels
    subset='validation'        # Use the validation subset of your data
)


Found 123 images belonging to 16 classes.
Found 26 images belonging to 16 classes.


In [8]:
## Use the same sequential model, but this time with augmented data

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Summary of the model architecture
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 196, 196, 32)      2432      
                                                                 
 max_pooling2d (MaxPooling2  (None, 98, 98, 32)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 94, 94, 32)        25632     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 47, 47, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 43, 43, 32)        25632     
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 21, 21, 32)        0

In [23]:
# Train the model
model.fit(
    train_generator,
    epochs=20,
    validation_data=validation_generator,
    callbacks = [early_stopping_monitor]
)


Epoch 1/20
4/4 [==============================] - 18s 5s/step - loss: 3.1579 - accuracy: 0.1545 - val_loss: 2.9714 - val_accuracy: 0.2308
Epoch 2/20
4/4 [==============================] - 15s 4s/step - loss: 2.2861 - accuracy: 0.2927 - val_loss: 2.8493 - val_accuracy: 0.3462
Epoch 3/20
4/4 [==============================] - 15s 4s/step - loss: 2.2170 - accuracy: 0.2927 - val_loss: 2.5735 - val_accuracy: 0.2692
Epoch 4/20
4/4 [==============================] - 15s 4s/step - loss: 2.1699 - accuracy: 0.2764 - val_loss: 2.5558 - val_accuracy: 0.3077
Epoch 5/20
4/4 [==============================] - 15s 4s/step - loss: 2.0440 - accuracy: 0.3171 - val_loss: 2.4582 - val_accuracy: 0.3462
Epoch 6/20
4/4 [==============================] - 15s 4s/step - loss: 1.9351 - accuracy: 0.3984 - val_loss: 2.6110 - val_accuracy: 0.3462
Epoch 7/20
4/4 [==============================] - 15s 4s/step - loss: 1.8352 - accuracy: 0.4309 - val_loss: 2.3791 - val_accuracy: 0.3462
Epoch 8/20
4/4 [==================

#### iii. Model 3 -- create a model with more layers
After building the base model and seeing that it performs poorly on the validation data, we'll add more layers and see how it does.

In [9]:
# Create the model
model = Sequential()

# Add a 2D Convolutional layer -- start with a small kernel size
model.add(Conv2D(32, kernel_size=(5, 5), padding='valid', activation='relu', input_shape=(image_size[0], image_size[1], 3)))

# Add a MaxPooling layer
model.add(MaxPooling2D(pool_size=(2, 2)))

# Add a 2D Convolutional layer -- start with a small kernel size
model.add(Conv2D(32, kernel_size=(5, 5), padding='valid', activation='relu', input_shape=(image_size[0], image_size[1], 3)))

# Add a MaxPooling layer
model.add(MaxPooling2D(pool_size=(2, 2)))

# Add a 2D Convolutional layer -- start with a small kernel size
model.add(Conv2D(32, kernel_size=(5, 5), padding='valid', activation='relu', input_shape=(image_size[0], image_size[1], 3)))

# Add a MaxPooling layer
model.add(MaxPooling2D(pool_size=(2, 2)))

# Add a dropout layer
model.add(Dropout(0.2))

# Flatten the output from Convolutional layers
model.add(Flatten())

# Add a fully connected Dense layer
model.add(Dense(128, activation='relu'))

# Add a fully connected Dense layer
model.add(Dense(50, activation='relu'))

# Add the final Dense layer with softmax activation (ensures predictions sum to 1)
model.add(Dense(num_classes, activation='softmax'))


# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Summary of the model architecture
model.summary()


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 196, 196, 32)      2432      
                                                                 
 max_pooling2d_3 (MaxPoolin  (None, 98, 98, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_4 (Conv2D)           (None, 94, 94, 32)        25632     
                                                                 
 max_pooling2d_4 (MaxPoolin  (None, 47, 47, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_5 (Conv2D)           (None, 43, 43, 32)        25632     
                                                                 
 max_pooling2d_5 (MaxPoolin  (None, 21, 21, 32)       

In [10]:
# Train the model
model.fit(
    train_generator,
    epochs=20,
    validation_data=validation_generator,
    callbacks = [early_stopping_monitor]
)


Epoch 1/20
4/4 [==============================] - 18s 5s/step - loss: 2.7493 - accuracy: 0.1382 - val_loss: 2.6140 - val_accuracy: 0.1538
Epoch 2/20
4/4 [==============================] - 15s 4s/step - loss: 2.6408 - accuracy: 0.1707 - val_loss: 2.5341 - val_accuracy: 0.2692
Epoch 3/20
4/4 [==============================] - 15s 4s/step - loss: 2.5538 - accuracy: 0.1789 - val_loss: 2.4970 - val_accuracy: 0.3077
Epoch 4/20
4/4 [==============================] - 15s 4s/step - loss: 2.4132 - accuracy: 0.2033 - val_loss: 2.4964 - val_accuracy: 0.1923
Epoch 5/20
4/4 [==============================] - 15s 4s/step - loss: 2.3037 - accuracy: 0.3008 - val_loss: 2.4680 - val_accuracy: 0.2308
Epoch 6/20
4/4 [==============================] - 15s 4s/step - loss: 2.2138 - accuracy: 0.3171 - val_loss: 2.4743 - val_accuracy: 0.2692
Epoch 7/20
4/4 [==============================] - 15s 4s/step - loss: 2.1587 - accuracy: 0.3171 - val_loss: 2.6220 - val_accuracy: 0.2308
Epoch 8/20
4/4 [==================

In [ ]:
# Plot the accuracy for the training and validation data

